In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from darts import TimeSeries

df = pd.read_csv('../csv/Ming19Jun67_total_order.csv')
df.tail()

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df['order_completed_at'] = pd.to_datetime(df['order_completed_at'])
# make timeseries Object
series = TimeSeries.from_dataframe(df,
                                   time_col='order_completed_at',
                                   value_cols=['unique_order_count'],
                                   freq="D")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

plt.figure(figsize=(20, 10)) 
series.plot()

plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
plt.gcf().autofmt_xdate() 

plt.title('Total Order By Date from 2022-2024')
plt.xlabel('Date')
plt.ylabel('Number of Orders (Unit)')

plt.show()

In [ ]:
# training, validation =series.split_before(0.9)
# training, validation =series.split_before(pd.Timestamp('2024-05-01')) # 1 month
training, validation =series.split_before(pd.Timestamp('2024-05-01')) # 2 month

validation_df = validation.pd_dataframe()
training_df = training.pd_dataframe()
print(training_df.tail())
print(validation_df.head())

In [ ]:
print(validation_df.shape)

In [ ]:
plt.figure(figsize=(20, 10))  # ปรับขนาดของภาพ (กว้าง x สูง)
training.plot()
validation.plot()

# ตั้งค่ารูปแบบของวันที่บนแกน x
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
plt.gcf().autofmt_xdate()  # ปรับให้วันที่ไม่ซ้อนกัน

# กำหนด Title และชื่อของแกน x และ y
plt.title('Total Order By Date from 2022-2024')
plt.xlabel('Date')
plt.ylabel('Number of Orders (Unit)')

plt.show()

In [ ]:
# from darts.models import Prophet
# from darts.metrics import mape, mae, mse, mase
# from math import sqrt
# from darts.utils.utils import ModelMode, SeasonalityMode

# parameters = {
#     "trend":[ModelMode.ADDITIVE, ModelMode.ADDITIVE],
#     "seasonal": [SeasonalityMode.ADDITIVE, SeasonalityMode.MULTIPLICATIVE],
#     'seasonal_periods': [4, 7, 12, 24, 30],
#     "random_state": [42]
# }

# best_parameter = Prophet.gridsearch(
#     parameters=parameters,
#     series = series, #note (training + validation) X only training set!!!
#     start=pd.Timestamp("2024-05-01"),
#     forecast_horizon=7,
#     stride=1,          
#     last_points_only=False,  
#     metric=mape,
#     verbose=-1
# )

In [ ]:
from darts.models import Prophet
from darts.metrics import mape, mae, mse, mase
from math import sqrt

model = Prophet(country_holidays="TH")
model.fit(training)

In [ ]:
from darts.metrics import mape, mae, mse, mase
from math import sqrt

backtest_errors = model.backtest(
     series,
     start=pd.Timestamp('2024-05-01'),          # เริ่มต้นที่ 90% ของข้อมูล
     forecast_horizon=7, #จำนวนก้าวการพยากรณ์ในอนาคตที่ต้องการ
     stride=1,           #ช่วงการเลื่อนในการทดสอบแต่ละครั้ง
     last_points_only=False,  #  ให้แสดงเฉพาะจุดสิ้นสุดของการพยากรณ์หรือไม่
     metric=mape,
     reduction=np.mean           #ฟังก์ชันการลดรูปผลลัพธ์ (เช่น mean การหาเฉลี่ย)
)

In [ ]:
print(f'backtest_errors MAPE: {backtest_errors}')

In [ ]:
# result from training , validation only
prediction = model.predict(len(validation))

mape_score = mape(validation, prediction)
print(f'score MAPE: {round(mape_score, 4)}')
print(f'score ACC: {100-(round(mape_score, 4))}')

mse_score = mse(validation, prediction)
print(f'score MSE: {round(mse_score, 4)}')

print(f'score RMSE: {round(sqrt(mse_score), 4)}')

mae_score = mae(validation, prediction)
print(f'score MAE: {round(mae_score, 4)}')

In [ ]:
plt.figure(figsize=(20, 10))  # ปรับขนาดของภาพ (กว้าง x สูง)
##prediction
prediction.plot(label="prediction")
training.plot(label="training")
validation.plot(label="validation")

plt.legend()
plt.title('Train, Validation, and Prediction')
plt.xlabel('Date')
plt.ylabel('Order Unit')
plt.show()

In [ ]:
# predict future
model.fit(series)

In [ ]:
#check date
series.pd_dataframe().tail(14)

In [ ]:
forecast = model.predict(7)
# forecast  = scaler.inverse_transform(forecast)
forecast = forecast.pd_dataframe().reset_index()
forecast.columns = ['Date', 'Total Order']
forecast.tail(7)

forecast.to_csv('../forecast/19-25-06-67_prophet.csv', index=False)

In [ ]:
model.save("../model/Prophet_19Jun67_1month.pkl")